In [1]:
import numpy as np
import ice_funcs as icef

## Todo:
- Make Bias adjustable to avoid division by zero (calculation of alpha).
- THERE MIGHT A PROBLEM WITH MY REFRESH... place a bomb
- Work on output of an array of ice positions that's timesliced according to global time
- What the fuck is wrong with sat opp?? (division by zero thing?)

## Backburner-type-beat:
- make sure everything is numbafied
- Continue commenting in `ice_funcs.py`


## Improvements
- Monopole matching
- correct geometry factors $G_b$ and $H_b$
- Make the refreshing process more efficient



In [2]:
# CONSTANTS

L = 101
W = (L+1)//2
INITIAL_SAT = 1

In [3]:
gu = icef.GeneralUtilities(101) # general utilities
pu = icef.PhysicsUtilities(0.01, 10, X_0=1) # physics utilities
sru = icef.SaturationRelaxationUtilities(101, 200)
gru = icef.GrowthUtilities(101)


In [4]:
neighbor_array_test = gu.neighbor_array
diffusion_rules_test = sru.diffusion_rules



print(diffusion_rules_test)

[[[-2.6711958e+07 -2.6711958e+07 -2.6711958e+07 -2.6711958e+07
   -2.6711958e+07 -2.6711958e+07]
  [-2.6711958e+07 -2.6711958e+07 -2.6711958e+07 -2.6711958e+07
   -2.6711958e+07 -2.6711958e+07]
  [-2.6711958e+07 -2.6711958e+07 -2.6711958e+07 -2.6711958e+07
   -2.6711958e+07 -2.6711958e+07]
  ...
  [-6.4341379e+22  3.0829968e-41  2.8025969e-45  1.9057659e-43
    0.0000000e+00  0.0000000e+00]
  [ 1.6507296e-42  0.0000000e+00  0.0000000e+00  0.0000000e+00
   -6.4379965e+22  3.0829968e-41]
  [ 2.8025969e-45  1.1210388e-44  5.6051939e-45  0.0000000e+00
   -6.5910361e+22  3.0829968e-41]]

 [[ 1.6666667e-01  3.3333334e-01  3.3333334e-01  1.6666667e-01
    0.0000000e+00  0.0000000e+00]
  [ 1.6666667e-01  1.6666667e-01  1.6666667e-01  1.6666667e-01
    1.6666667e-01  1.6666667e-01]
  [ 1.6666667e-01  1.6666667e-01  1.6666667e-01  1.6666667e-01
    1.6666667e-01  1.6666667e-01]
  ...
  [ 1.6666667e-01  1.6666667e-01  1.6666667e-01  1.6666667e-01
    1.6666667e-01  1.6666667e-01]
  [ 1.6666667e-0

In [5]:
default_ice_map = np.full((101, 101), False)
default_ice_map[-1,0] = True
default_ice_map[-2,0] = True
default_ice_map[-3,0] = True
default_ice_map[-3,1] = True

# %timeit test_boundary_map
boundary_map_test = gu.construct_boundary_map(default_ice_map)
print(boundary_map_test[-5:,:5])

sat_map_test = np.full((101,101), 1, dtype=np.float32)

normal_cells_test, boundary_cells_test = gu.distinguish_cells(default_ice_map, boundary_map_test)

opp_array_test = gu.construct_opp_array(boundary_cells_test, default_ice_map, neighbor_array_test)

[[0 0 0 0 0]
 [1 2 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]


In [6]:
# pu.apply_boundary_condition(2, 2, sat_map_test, 0.5, boundary_map_test)

sat_map_test = np.full((101,101), 1, dtype=np.float32)

new_thang = sru.diffuse_to_convergence(sat_map_test, 0.0001, normal_cells_test, boundary_cells_test, boundary_map_test, neighbor_array_test, opp_array_test, pu)
gru.update_minimum_time(boundary_cells_test, sat_map_test, boundary_map_test, pu)
gru.update_filling_array(boundary_cells_test, sat_map_test, boundary_map_test, pu)

print(new_thang[1])
print(new_thang[2])
print(new_thang[0][-10:,:5])


True
50
[[0.99865556 0.99835116 0.99807584 0.99787086 0.99777734]
 [0.99790716 0.99746865 0.99709535 0.9968527  0.99677205]
 [0.9968196  0.99620247 0.9957176  0.995485   1.        ]
 [0.9952684  0.9944138  0.99380445 0.99358547 1.        ]
 [0.9930891  0.991905   0.9912093  1.         1.        ]
 [0.99010193 0.98826534 0.98723716 1.         1.        ]
 [0.98661655 0.9819473  1.         1.         1.        ]
 [1.         1.         1.         1.         1.        ]
 [1.         1.         1.         1.         1.        ]
 [1.         1.         1.         1.         1.        ]]


In [7]:
SnowflakeSim = icef.SnowflakeSimulation(501, 200)

print("Strating simulation")

final_ice_array = SnowflakeSim.run_simulation(10, 0.001)
np.savetxt("Example_ice_map", final_ice_array,)

Strating simulation
